## GoogleNet Implementation

(https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43022.pdf)

- 2014 ILSVRC 1nd place 
- Inception Module

![alt text](./networks/googlenet_diagram.png)

-Auxiliary classification output은 구현 X

## 1. Settings
### 1) Import required libraries

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

### 2) Hyperparameter

In [5]:
batch_size= 1
learning_rate = 0.0002
epoch = 100

## 2. Data Loader

In [6]:
img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=2)

C:\Users\user\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:563: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  "please use transforms.RandomResizedCrop instead.")


## 3. Model 
### 1) Base module

![](https://user-images.githubusercontent.com/36406676/53284616-22368380-379a-11e9-87ce-e0b231c91bf6.jpg)

In [20]:
def conv_1(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,1,1), 
        ## (input_dimension, output_dimension, stride, padding)
        
        nn.ReLU(),
    )
    return model

def conv_1_3(in_dim, mid_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, mid_dim, 1,1),      
        ## 1X1 filter with 1 stride
        
        nn.ReLU(),
        nn.Conv2d(mid_dim, out_dim,3,1,1),
        ## 3X3 filter with 1 stride , 1 padding
        
        nn.ReLU()
        )
    return model

def conv_1_5(in_dim, mid_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim, mid_dim,1,1),
        nn.ReLU(),
        nn.Conv2d(mid_dim, out_dim,5,1,2),
        ## 5X5 filter with 1 stride, 2 padding
        
        nn.ReLU()
    )
    return model

def max_3_1(in_dim, out_dim):
    model = nn.Sequential(
        nn.MaxPool2d(3,1,1),
        ## Maxpooling 3X3 filter with 1 stride, 1 padding
        
        nn.Conv2d(in_dim, out_dim,1,1),
        nn.ReLU()
    )
    return model

### 2) Inception Module

![](https://user-images.githubusercontent.com/36406676/53285302-501fc600-37a2-11e9-948a-adde14860221.jpg)

In [21]:
class inception_module(nn.Module):
    
    def __init__(self,in_dim,out_dim_1,mid_dim_3,out_dim_3,mid_dim_5,out_dim_5,pool):
        super(inception_module,self).__init__()
        
        self.conv_1 = conv_1(in_dim,out_dim_1)
        self.conv_1_3 = conv_1_3(in_dim, mid_dim_3,out_dim_3)
        self.conv_1_5 = conv_1_5(in_dim,mid_dim_5,out_dim_5)
        self.max_3_1 = max_3_1(in_dim,pool)
        
    def forward(self,x):
        out_1 = self.conv_1(x)
        out_2 = self.conv_1_3(x)
        out_3 = self.conv_1_5(x)
        out_4 = self.max_3_1(x)
        output = torch.cat([out_1, out_2, out_3, out_4],1)
        
        return output
        

### 3) GoogleNet

In [37]:
class GoogleNet(nn.Module):
    def __init__(self, base_dim, num_classes=2):
        super(GoogleNet,self).__init__()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3,base_dim,7,2,3),
            ## (64)
            nn.MaxPool2d(3,2,1),
            ## (64)
            nn.Conv2d(base_dim,base_dim*3,3,1,1),
            ## (192)
            nn.MaxPool2d(3,2,1)
        )
        
        self.layer_2 = nn.Sequential(
                inception_module(base_dim*3,64,96,128,16,32,32), 
                ## base_dim= 64, base*dim = 192 -> 64 + 128 + 32 +32 = 256 dimension
                inception_module(base_dim*4,128,128,192,32,96,64),
                ##  192 -> 128 + 192 + 96 + 64 = 480 dimension
                nn.MaxPool2d(3,2,1)
        )
        
        self.layer_3 = nn.Sequential(
                inception_module(480,192,96, 208,16,48,64),
                inception_module(512,160,112,224,24,64,64),
                inception_module(512,128,128,256,24, 64,64),
                inception_module(512,112,144,288,32,64,64),
                inception_module(528,256,160,320,32,128,128),
            nn.MaxPool2d(3,2,1)
        )
        
        self.layer_4 = nn.Sequential(
            inception_module(832,256,160,320,32,128,128),
            inception_module(832,384,192,384,48,128,128),
            nn.AvgPool2d(7,1),
        )
        
        self.layer_5 = nn.Dropout2d(0.4)
        self.fc_layer = nn.Linear(1024,1000)
        ## Label 1000개
        
    def forward(self,x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = self.layer_5(out)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
            
        return out
        
model = GoogleNet(base_dim = 64)
for i in model.named_children():
    print(i)

('layer_1', Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
))
('layer_2', Sequential(
  (0): inception_module(
    (conv_1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
    )
    (conv_1_3): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
    )
    (conv_1_5): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (3): ReLU()
    )
    (max_3_1): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=1, padding=1,

## 4. Optimizer & Loss

In [38]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## 5. Train

In [39]:
for i in range(epoch):
    for img, label in img_batch:
        img = Variable(img)
        label = Variable(label)
                         
        optimizer.zero_grad()
        output = model(img)
        
        loss = loss_func(output,label)
        loss.backward()
        optimizer.step()
        
    if i % 10 ==0:
        print(loss)

tensor(6.8779, grad_fn=<NllLossBackward>)
tensor(2.0848, grad_fn=<NllLossBackward>)
tensor(1.2593, grad_fn=<NllLossBackward>)
tensor(0.6758, grad_fn=<NllLossBackward>)
tensor(0.6851, grad_fn=<NllLossBackward>)
tensor(1.0438, grad_fn=<NllLossBackward>)
tensor(1.0930, grad_fn=<NllLossBackward>)
tensor(0.5431, grad_fn=<NllLossBackward>)
tensor(1.1745, grad_fn=<NllLossBackward>)
tensor(0.1984, grad_fn=<NllLossBackward>)
